In [15]:
# Numpy
import numpy as np
# Chainer
import chainer
import chainer.links as L
import chainer.functions as F
from chainer import Chain, optimizers, Variable
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

In [16]:
X_train = ((1,2,3,4,5,6,7,8,9,10),(10,9,8,7,6,5,4,3,2,1),(1,2,3,4,5,6,7,8,9,10),(10,9,8,7,6,5,4,3,2,1),(1,2,3,4,5,6,7,8,9,10),(10,9,8,7,6,5,4,3,2,1))
T_train = ((1,0,0,0,0,0,0,0,0,1,0,1,0,0,0),(0,0,0,0,0,0,0,1,0,0,0,1,0,0,1),(1,0,0,0,0,0,0,0,0,1,0,1,0,0,0),(0,0,0,0,0,0,0,1,0,0,0,1,0,0,1),(1,0,0,0,0,0,0,0,0,1,0,1,0,0,0),(0,0,0,0,0,0,0,1,0,0,0,1,0,0,1))
X_test = ((1,3,4,4,5,6,7,8,9,10),(1,3,4,7,6,5,4,3,2,1))
T_test = ((1,0,0,0,0,0,0,0,0,1,0,1,0,0,0),(0,0,0,0,0,0,0,1,0,0,0,1,0,0,1))

In [17]:
print(X_train)
print(T_train)

((1, 2, 3, 4, 5, 6, 7, 8, 9, 10), (10, 9, 8, 7, 6, 5, 4, 3, 2, 1), (1, 2, 3, 4, 5, 6, 7, 8, 9, 10), (10, 9, 8, 7, 6, 5, 4, 3, 2, 1), (1, 2, 3, 4, 5, 6, 7, 8, 9, 10), (10, 9, 8, 7, 6, 5, 4, 3, 2, 1))
((1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1), (1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1), (1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1))


In [18]:
print(T_test)

((1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1))


In [19]:
x_train = np.array(X_train, dtype=np.float32)
x_test = np.array(X_test, dtype=np.float32)
t_train = np.array(T_train, dtype=np.int32)
t_test = np.array(T_test, dtype=np.int32)

In [20]:
print(x_test[0:5])

[[ 1.  3.  4.  4.  5.  6.  7.  8.  9. 10.]
 [ 1.  3.  4.  7.  6.  5.  4.  3.  2.  1.]]


In [21]:
print(x_train[1].ndim)

1


In [22]:
# DNNのクラス
class DNN(Chain):
    def __init__(self):
        super(DNN, self).__init__(
            l1 = L.Linear(10, 100),
            l2 = L.Linear(100, 100),
            l3 = L.Linear(100, 15)
        )
    def forward(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        h3 = self.l3(h2)
        return h3

# DNNクラスのインスタンス
model = DNN()

In [23]:
# 最適化手法
optimizer = optimizers.Adam()
optimizer.setup(model)

# エポック数
n_epoch = 1

# バッチサイズ
batch_size = 1

In [24]:
# 学習実行
for epoch in range(n_epoch):
    sum_loss = 0
    perm = np.random.permutation(2)
    for i in range(0, 2, batch_size):
        x = Variable(x_train[perm[i:i+batch_size]])
        t = Variable(t_train[perm[i:i+batch_size]])
        y = model.forward(x)
        model.cleargrads()
        loss = F.softmax_cross_entropy(y, t)
        loss.backward()
        optimizer.update()
        sum_loss += loss.data*batch_size

    print("epoch: {}, mean loss: {}".format(epoch, sum_loss/2))

InvalidType: 
Invalid operation is performed in: SoftmaxCrossEntropy (Forward)

Expect: in_types[1].ndim == in_types[0].ndim - 1
Actual: 2 != 1

In [84]:
# テスト実行
cnt = 0
for i in range(10000):
    x = Variable(np.array([x_test[i]], dtype=np.float32))
    t = t_test[i]
    y = model.forward(x)
    y = np.argmax(y.data[0])
    if t == y:
        cnt += 1

# 正解数と正解率を表示
print(cnt)
print("accuracy: {}".format(cnt/(10000)))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [16]:
y = model.forward(x_test)

In [17]:
print(y[1:5])

variable([[ -0.854458     0.21955684]
          [-30.156626    30.416458  ]
          [ -2.1364415    1.1742759 ]
          [ -3.090154     2.1957014 ]])


In [18]:
from chainer import serializers
serializers.save_npz("mymodel0803_5.npz", model)

In [19]:
print(x_test[0:5])

[[2. 2. 0. 0. 1. 0. 1. 0. 1. 0. 5.]
 [2. 2. 1. 0. 0. 0. 0. 0. 0. 0. 4.]
 [2. 2. 0. 0. 2. 0. 0. 0. 0. 0. 3.]
 [2. 2. 1. 0. 0. 0. 0. 0. 1. 0. 6.]
 [1. 2. 1. 0. 0. 1. 0. 0. 0. 0. 4.]]
